![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

# 24.1 Improved_Entity_Resolution_with_SentenceChunkEmbeddings

Enrich embeddings of chunks to get more accurate resolution results


In [1]:
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

Saving jsl_keys.json to jsl_keys (3).json


In [2]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

In [3]:
import json
import os
import pandas as pd
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
from sparknlp.util import *
import sparknlp_jsl
import sparknlp

from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

Spark NLP Version : 3.2.1
Spark NLP_JSL Version : 3.2.0rc3


<h1>!!! Warning !!!</h1>

**If you get an error related to Java port not found 55, it is probably because that the Colab memory cannot handle the model and the Spark session died. In that case, try on a larger machine or restart the kernel at the top and then come back here and rerun.**

## ICD10 CM Resolver Pipeline

In [4]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("documents")
    
sentence_detector = SentenceDetector()\
    .setInputCols("documents")\
    .setOutputCol("sentences")
    
tokenizer = Tokenizer()\
    .setInputCols("sentences")\
    .setOutputCol("token")
    
embeddings = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", 'en', 'clinical/models')\
    .setInputCols(["sentences","token"])\
    .setOutputCol("embeddings")
    
ner = MedicalNerModel()\
    .pretrained("ner_jsl_greedy", "en", "clinical/models")\
    .setInputCols("sentences", "token","embeddings")\
    .setOutputCol("ner")
    
ner_converter = NerConverter()\
    .setInputCols("sentences", "token", "ner")\
    .setOutputCol("ner_chunk").setWhiteList(['Symptom', 'Diabetes', 'Disease_Syndrome_Disorder',
                                            'Oncological', 'Injury_or_Poisoning'])

#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = BertSentenceChunkEmbeddings\
    .pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentences", "ner_chunk"])\
    .setOutputCol("sentence_chunk_embeddings").setChunkWeight(0.5) #default : 0.5

resolver = SentenceEntityResolverModel.pretrained('sbiobertresolve_icd10cm', 'en', 'clinical/models')\
            .setInputCols(["ner_chunk", "sentence_chunk_embeddings"]) \
            .setOutputCol("resolution")\
            .setDistanceFunction("EUCLIDEAN")

def create_pipeline_model():
    pipeline = Pipeline().setStages(
    [
        documenter,
        sentence_detector, 
        tokenizer, 
        embeddings, 
        ner, 
        ner_converter, 
        sentence_chunk_embeddings,
        resolver
    ])
    
    return LightPipeline(pipeline.fit(spark.createDataFrame([['']]).toDF("text")))

l_model = create_pipeline_model()

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_jsl_greedy download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm download started this may take some time.
Approximate size to download 199 MB
[OK!]


**Helper function to process output**

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

def print_result(res):
    def get_res(col):
        dists = col.metadata['all_k_cosine_distances'].split(':::')[:5]
        resos = col.metadata['all_k_resolutions'].split(':::')[:5]
        reses = col.metadata['all_k_results'].split(':::')[:5]
        return pd.DataFrame({'code': reses, 'resolutions': resos , 'dist': dists})
    df_arr = []
    for chunk, resol in zip(res['ner_chunk'], res['resolution']):        
        t_df = get_res(resol)
        t_df['chunk'] = chunk.result
        t_df['entity'] = chunk.metadata['entity']
        
        df_arr.append(t_df)
        
    return pd.concat(df_arr, axis=0) 


**Explanation:**

Even though the chunk itself does not specify right leg, we can still get that information from surrounding text

The "setChunkWeight" parameter has a default value of 0.5 - meaning taking equal representation of the cunk and sentence. Setting it to 1.0 would mean discarding the context information completely and only relying on the chunk, while a value of 0.0 would mean only relying on information in the sentence, not the chunk.

**Looking at top 5 results for analysis**

In [6]:
sentence_chunk_embeddings.setChunkWeight(0.5)
l_model = create_pipeline_model()

text = "She complains of pain in right leg."

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,M79661,Pain in right lower leg,0.0698,pain,Symptom
1,R072,Precordial pain,0.0708,pain,Symptom
2,R1031,Right lower quadrant pain,0.0737,pain,Symptom
3,R1011,Right upper quadrant pain,0.0721,pain,Symptom
4,R1033,Periumbilical pain,0.0783,pain,Symptom


Relying only on chunk - we get less specific results

In [7]:
sentence_chunk_embeddings.setChunkWeight(1.0)
l_model = create_pipeline_model()

text = "She complains of pain in right leg."

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,R072,Precordial pain,0.0795,pain,Symptom
1,R6884,Jaw pain,0.0799,pain,Symptom
2,R1033,Periumbilical pain,0.0825,pain,Symptom
3,R141,Gas pain,0.0884,pain,Symptom
4,R0782,Intercostal pain,0.0919,pain,Symptom


Using the entire sentence to get embedding

In [8]:
sentence_chunk_embeddings.setChunkWeight(0.0)
l_model = create_pipeline_model()

text = "She complains of pain in right leg."

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,M79661,Pain in right lower leg,0.0914,pain,Symptom
1,I83811,Varicose veins of right lower extremity with pain,0.0989,pain,Symptom
2,M79604,Pain in right leg,0.0984,pain,Symptom
3,M79651,Pain in right thigh,0.1039,pain,Symptom
4,R1011,Right upper quadrant pain,0.1049,pain,Symptom


More elaborate example

In [9]:


sentence_chunk_embeddings.setChunkWeight(0.5)
l_model = create_pipeline_model()

text = """A 20 year old female patient badly tripped while going down stairs. She complains of right leg pain. 
Her x-ray showed right hip fracture. Hair line fractures also seen on the left knee joint.
She also suffered from trauma and slight injury on the head.

OTHER CONDITIONS: She was also recently diagnosed with diabetes, which is of type 2.
"""

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,M79661,Pain in right lower leg,0.0667,leg pain,Symptom
1,R1031,Right lower quadrant pain,0.0706,leg pain,Symptom
2,R1012,Left upper quadrant pain,0.0705,leg pain,Symptom
3,I83811,Varicose veins of right lower extremity with pain,0.0795,leg pain,Symptom
4,G546,Phantom limb syndrome with pain,0.0776,leg pain,Symptom
0,M84451S,"Pathological fracture, right femur, sequela",0.1119,hip fracture,Injury_or_Poisoning
1,M84351S,"Stress fracture, right femur, sequela",0.1301,hip fracture,Injury_or_Poisoning
2,M84459P,"Pathological fracture, hip, unspecified, subsequent encounter for fracture with malunion",0.1430,hip fracture,Injury_or_Poisoning
3,M84451A,"Pathological fracture, right femur, initial encounter for fracture",0.1392,hip fracture,Injury_or_Poisoning
4,M84421S,"Pathological fracture, right humerus, sequela",0.1463,hip fracture,Injury_or_Poisoning


## CPT Resolver

In [10]:
resolver = SentenceEntityResolverModel.pretrained('sbiobertresolve_cpt_procedures_augmented', 'en', 'clinical/models')\
            .setInputCols(["ner_chunk", "sentence_chunk_embeddings"]) \
            .setOutputCol("resolution")\
            .setDistanceFunction("EUCLIDEAN")

ner_converter.setWhiteList(['Procedure', 'Test'])

sbiobertresolve_cpt_procedures_augmented download started this may take some time.
Approximate size to download 78.3 MB
[OK!]


NerConverter_aa29d6b756fa

Using context to get specific results

In [11]:
sentence_chunk_embeddings.setChunkWeight(0.5)
l_model = create_pipeline_model()

text = "extraction of cyst in kidney"

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,50280,Decortication of cyst of kidney [Excision or unroofing of cyst(s) of kidney],0.0877,extraction of cyst,Procedure
1,50290,Excision of perinephric cyst (procedure) [Excision of perinephric cyst],0.1048,extraction of cyst,Procedure
2,50390,"Therapeutic aspiration of renal cyst (procedure) [Aspiration and/or injection of renal cyst or pelvis by needle, percutaneous]",0.1151,extraction of cyst,Procedure
3,68020,"Incision of conjunctiva with drainage of cyst (procedure) [Incision of conjunctiva, drainage of cyst]",0.1405,extraction of cyst,Procedure
4,62162,"Excision of cyst of brain meninges (procedure) [Neuroendoscopy, intracranial; with fenestration or excision of colloid cyst, including placement of external ventricular catheter for drainage]",0.1788,extraction of cyst,Procedure


Using ner chunk only - results are generic

In [12]:
sentence_chunk_embeddings.setChunkWeight(1.0)
l_model = create_pipeline_model()

text = "extraction of cyst in kidney"

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,68020,"Incision of conjunctiva with drainage of cyst (procedure) [Incision of conjunctiva, drainage of cyst]",0.1411,extraction of cyst,Procedure
1,50290,Excision of perinephric cyst (procedure) [Excision of perinephric cyst],0.1537,extraction of cyst,Procedure
2,25145,"Excision of sequestrum [Sequestrectomy (eg, for osteomyelitis or bone abscess), forearm and/or wrist]",0.1592,extraction of cyst,Procedure
3,60200,"Excision of cyst of thyroid (procedure) [Excision of cyst or adenoma of thyroid, or transection of isthmus]",0.1679,extraction of cyst,Procedure
4,39200,Resection of mediastinal cyst [Resection of mediastinal cyst],0.1678,extraction of cyst,Procedure


## RxNorm Resolver

In [13]:
resolver = SentenceEntityResolverModel.pretrained('sbiobertresolve_rxnorm', 'en', 'clinical/models')\
            .setInputCols(["ner_chunk", "sentence_chunk_embeddings"]) \
            .setOutputCol("resolution")\
            .setDistanceFunction("EUCLIDEAN")

ner_converter.setWhiteList(['Drug'])

sbiobertresolve_rxnorm download started this may take some time.
Approximate size to download 802.6 MB
[OK!]


NerConverter_aa29d6b756fa

Even though the chunk only contains drug name, we can still more accurate code by looking at the form of drug

In [14]:
sentence_chunk_embeddings.setChunkWeight(0.5)
l_model = create_pipeline_model()

text = "rituximab in the form of injection"

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,1657861,rituximab injection,0.0268,rituximab,Drug
1,121191,rituximab,0.0460,rituximab,Drug
2,1657863,rituximab injection [rituxan],0.0543,rituximab,Drug
3,1157967,rituximab injectable product,0.0548,rituximab,Drug
4,2273515,rituximab injection [ruxience],0.0593,rituximab,Drug


In [15]:
sentence_chunk_embeddings.setChunkWeight(1.0)
l_model = create_pipeline_model()

text = "rituximab in the form of injection"

result = l_model.fullAnnotate(text)[0]

print_result(result)

,code,resolutions,dist,chunk,entity
0,121191,rituximab,0.0000,rituximab,Drug
1,1657861,rituximab injection,0.0451,rituximab,Drug
2,2105824,rituximab-abbs,0.0549,rituximab,Drug
3,226754,rituxan,0.0556,rituximab,Drug
4,318341,cetuximab,0.0675,rituximab,Drug


## SNOMED Resolver

In [16]:
resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["ner_chunk", "sentence_chunk_embeddings"]) \
      .setOutputCol("resolution")\
      .setDistanceFunction("EUCLIDEAN")

ner_converter.setWhiteList(['Drug', 'Symptom', 'Diabetes', 'Disease_Syndrome_Disorder',
                                            'Oncological', 'Injury_or_Poisoning', 'Procedure', 'Treatmentb'])

sbiobertresolve_snomed_findings download started this may take some time.
Approximate size to download 1.3 GB
[OK!]


NerConverter_aa29d6b756fa

In [17]:

sentence_chunk_embeddings.setChunkWeight(0.5)
l_model = create_pipeline_model()

text = """A 20 year old female patient badly tripped while going down stairs. She complains of right leg pain. 
Her x-ray showed right hip fracture. Hair line fractures also seen on the left knee joint.
She also suffered from trauma and slight injury on the head.

OTHER CONDITIONS: She was also recently diagnosed with diabetes, which is of type 2.

She we through 
"""

result = l_model.fullAnnotate(text)[0]

print_result(result)


,code,resolutions,dist,chunk,entity
0,203124006,leg pain,0.0395,leg pain,Symptom
1,316941000119108,pain of right lower leg,0.0525,leg pain,Symptom
2,12247531000119106,bilateral lower leg pain,0.0545,leg pain,Symptom
3,203127004,pain in leg,0.0580,leg pain,Symptom
4,5771000119106,phantom limb syndrome with pain,0.0576,leg pain,Symptom
0,208520007,hip fracture,0.0646,hip fracture,Injury_or_Poisoning
1,263225007,hip fracture,0.0646,hip fracture,Injury_or_Poisoning
2,11315041000119101,pathological fracture of right hip,0.0694,hip fracture,Injury_or_Poisoning
3,704063001,stress fracture of hip,0.0722,hip fracture,Injury_or_Poisoning
4,5913000,fracture of hip,0.0751,hip fracture,Injury_or_Poisoning
